# Regularization

If we suspect that we have a high variance problem, one of the thing that we can do is get more training data, but at times, it is not possible to get more data. So instead, adding regularization will often help to prevent overfitting, or to reduce variance in your network.

Let's develop idea around **Logistic Regression**.

We had the cost function as $J(w,b) = \frac{1}{m} \sum_{i=1}^{m} L(\hat{y}^{(i)}, y^{(i)})$ and we want to minimize it as $\underset{w,b}{\text{min}} J(w,b)$ where $w$ is an x-dimensional parameter vector ($w \in R^{n_x}$), and $b$ is a real number.

To add regularization to logistic regression, we add lambda which is called the regularziation parameter.
$$ J(w,b) = \frac{1}{m} \sum_{i=1}^{m} L(\hat{y}^{(i)}, y^{(i)}) + \frac{\lambda}{2m} ||w^2||_2 $$
where $||w^2||_2 = \sum_{j=1}^{n_x} w_j^2 = w^T\cdot w $

The above is called the $L_2$ regularization.

We also have $L_1$ regularization, when we add the term: $\frac{\lambda}{2m} \sum_{i=1}^{m}|w| = \frac{\lambda}{2m} \sum_{i=1}^{m}||w||_1$

If we use $L_1$ regularization, then $w$ will end up being sparse, i.e. $w$ will have a lot of zeroes.

$\lambda$ is another hyperparameter that you might have to tune. For the programming exercises in Python, the keyword lambda is a reserved keyword. So we will use lambd instead. 

Now lets develop the idea of regularization around the **Neural Networks**

We have the cost function as $J(w^{[1]},b^{[1]}, \ldots , w^{[L]}, b^{[L]}) = \frac{1}{m} \sum\limits_{i=1}^{m} L(\hat{y}^{(i)}, y^{(i)}) + \frac{\lambda}{2m} \sum\limits_{i=1}^{L} ||w^{[L]}||^2 $ where $||w^{[L]}||^2 = \sum\limits_{i=1}^{n^{[l]}} \sum\limits_{j=1}^{n^{[l-1]}} (w_{i,j}^{[l]} )^2 $

This matrix norm $||w^{[l]}||^2$ is called the Frobenius Norm of a Matrix, denoted F in the subscript.

**How do we implement Gradient Descent with this?**

Previously, we would complete $dw$ using backprop, where backprop would give us the partial derivative of J with respect to w.

$dw = (\text{from BackProp})\qquad \frac{ \partial J }{ \partial w^{[l]} }$

Then we update $w^{[l]} = w^{[l]} - \alpha \cdot dw^{[l]} $.

Now we will add the regularization terms to the above equations to compensate for the regularziation parameters.

$dw = (\text{from BackProp}) + \frac{\lambda}{2m} w^{[l]} \qquad \frac{ \partial J }{ \partial w^{[l]} }$

$L_2$ regularization is called weight decay.

# Why Regularization Reduces Overfitting?

In other words, why is it that shrinking or reducing the Frobenius norm with the parameters might cause less overfitting? 


One intuition is that if we set $\lambda = $ really big, then we will get $w^{[L]} \approx 0$. Further, this will take us from the case of overfitting to underfitting (high variance $\to$ high bias). Thus, we want to find an intermediate value of $\lambda$ so that final fitting of the datast is "just right" case (from Practical Aspect of Deep Learning first figure).  

Here is the reason why reducing the Frobenius norm will reduce overfitting: 

- Reducing Frobenius norm is equivalent to increase the value of $\lambda$ and decreasing the value of $w^{[l]}$
- Since $z^{[l]} = w^{[l]}a^{[l-1]} + b^{[l]}$, reducing the value of $w^{[l]}$ means reducing the value of $z^{[l]}$ (ignoring the effects of $b^{[l]}$). In other words, $z^{[l]}$ will take on relatively small number of values.
- If we use the activation function $g(z) = \text{tanh}(z)$, then with respect to $z^{[l]}$ it will become linear near the origin (around smaller values).
- Thus our whole Neural Network will be computing a network which is close to being a linear network (which is pretty simple funcion, not complex).
- Linear Functions are safe from overfitting the data, thus our overfitting problem is solved.

# Dropout Regularization 

There is another kind of regularization beside $L_1$ and $L_2$ regularization, called Dropout Regularization. With dropout, while going through each of the layers of the network and set some probability of eliminating a node in neural network. For example 

<img src="images/Before_Dropout_Regularization.jpeg" style="width:850px;height:300px;">
<caption><center><font><b>Figure 1</b>: Before Dropout Regularization.</font></center></caption>

<img src="images/After_dropout_regularization.jpeg" style="width:850px;height:400px;">
<caption><center><font><b>Figure 2</b>: After Dropout Regularization.</font></center></caption>

In the above, we set a probability of 0.5 for eliminating the nodes from the network, and thus half of the nodes were eliminated. We can see that we will end up with a much smaller and diminshed network.

This might seem like we are killing nodes at random, but this actually works!! 

### Implementing Dropout ("Inverted Dropout")

Lets choose number of layers, $l = 3$.

Keep prob is the probability that a given hidden unit will be kept. If keep-prob = 0.8, then this means that there is a 0.2 chance of eliminating any hidden unit.

Dropout vector for layer 3, $d3 = \text{np.random.rand(ai.shape[0]), ai.shape[1] } < \text{keep prob}$

- d3 generates a random matrix, where each example and for each hidden unit, there is a 0.8 chance that $d3 = 1$, and $20%$ chance that $d3 = 0.$

- In Python, d3 will be a boolean array where value is either True or False, rather than 1 and 0. The multiplication operation will work with the boolean array as it will interpret the True and False values as 1 and 0.

After this, we are going to equate the acitivations of that layer as $a3 = \text{np.multiply(a3, d3)} $. This is an element-wise multiplication. This multiplication ends up zeroing out the elements who are already zero in d3.


Now, we are going to take $a3$, and will scale it up by dividing by 0.8 (keep-prob). $a3\ /=\text{keep-prob}$. 
- Since $z^{[4]} = W^{[4]} \cdot a^{[3]} + b^{[4]} $, we reduced the value of $a^{[3]}$ by 20% (20% of the elements of $a^{[3]}$ will be zerod out). So, in order to not reduce the expected value of $z^{[4]}$, we perform the above (last) equation. That will bump the $a^{[3]}$ by roughly $20%$ and will not change the expected value of $a3.$
- This last scaling portion is called the **Inverted dropout** technique.

### Making predictions at test time

Given $x$ on which we want to make the prediction. $a^{[0]} = X$. 
- At test time, we are not going to use the dropout techniques!!! 

### Other Regularization Techniques

- Data Augmentation: Let us say we are buidling a cat classifier and we do not have enough data to train our algorithm, one thing we can do is flip the pictures so that we can get more pictures to train the algorithm on. We can also take random crop of the pictures for example zooming into the picture at an angle.

- Early Stopping: As we run the gradient descent, we are going to plot either the training error or the cost function vs number of itrations. It should decrease monotonically. In Early Stopping, along with the previous graph, we also plot the dev set error. This could be a classification error in a development sense or something like cost function like logistic loss or the log loss of the dev set. What we will find is that the dev set error will usually go down for a while and then it will increase from at one point. In early stopping, we will stop training the neural network where the dev set error starts to increase.